# <br><br> Download price data from Binance

Using the historical snapshots of coins ranked by market cap, we now retrieve their historical daily price data from Binance.

For each coin, we check whether it is listed on Binance — either in the **spot** market or as a **perpetual futures (perp)** contract. If available, we fetch the OHLC data using Binance’s public API.

This step builds a dataset of daily prices for the most relevant and liquid cryptocurrencies over time, aligned with their presence in CoinMarketCap's top rankings.

Coins that are not listed on Binance are skipped automatically.

# <br><br> Imports

In [1]:
from requests import get
import pandas as pd
import numpy as np

import time
import sys
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# <br><br> Built-in functions

In [2]:
from typing import List

def get_historical_klines(symbol: str, start_date: float, interval: str = '1h', spot: bool = True) -> pd.DataFrame:
    """
    Retrieve historical OHLC data from Binance Spot or Futures (Perpetual) API.

    Parameters
    ----------
    symbol : str
        Trading pair symbol (e.g., 'BTCUSDT').
    start_date : float
        Start date as a Unix timestamp in seconds.
    interval : str, optional
        Kline interval (default is '1h').
    spot : bool, optional
        Market type: True if Spot, False if Perp (default is True).

    Returns
    -------
    pd.DataFrame
        DataFrame with historical OHLCV data and calculated CVD approximation.
    """
    start_ms = int(start_date * 1e3)
    end_ms = int((time.time() - 300) * 1e3)  # 5 min buffer

    base_url = 'https://api4.binance.com/api/v3/klines' if spot else 'https://fapi.binance.com/fapi/v1/klines'

    all_data = []
    current_start = start_ms
    max_retries = 3

    def fetch_klines(start_ts: int, end_ts: int) -> pd.DataFrame:
        url = f"{base_url}?symbol={symbol}&interval={interval}&startTime={start_ts}&endTime={end_ts}&limit=1000"
        for attempt in range(max_retries):
            try:
                res = np.array(get(url).json())
                if len(res) == 0:
                    return pd.DataFrame()
                df = pd.DataFrame({
                    'timestamp': res[:, 0],
                    'open': res[:, 1],
                    'high': res[:, 2],
                    'low': res[:, 3],
                    'close': res[:, 4],
                }).astype(float)
                return df
            except Exception as e:
                if attempt == max_retries - 1:
                    raise Exception(f"Failed to fetch data after {max_retries} attempts.") from e
                time.sleep(5)

    # Main download loop
    while current_start < end_ms:
        batch = fetch_klines(current_start, end_ms)
        if batch.empty:
            break
        all_data.append(batch)
        current_start = int(batch['timestamp'].max()) + 1
        if len(batch) < 1000:
            break

    df = pd.concat(all_data).drop_duplicates('timestamp')
    df.sort_values(by='timestamp', inplace=True)
    df['timestamp'] /= 1e3
    df.index = range(len(df))

    return df


def get_binance_coins(spot: bool = True) -> List[str]:
    """
    Retrieve a list of USDT trading pairs (base assets) from Binance Spot or Perpetual Futures markets.

    Parameters
    ----------
    spot : bool, optional
        The market type to query: True if Spot or False if Perp (default is True).

    Returns
    -------
    List[str]
        List of base asset symbols in lowercase (e.g., ['btc', 'eth', 'sol', ...]).

    Raises
    ------
    ValueError
        If an unsupported market type is provided.
    """
    url = "https://api.binance.com/api/v3/ticker/price" if spot else "https://fapi.binance.com/fapi/v1/ticker/price"

    response = get(url).json()
    usdt_pairs = [ticker['symbol'] for ticker in response if 'USDT' in ticker['symbol']]
    base_assets = [pair.replace('USDT', '').lower() for pair in usdt_pairs]

    return base_assets

# <br><br> Download data

In [3]:
import os
from datetime import datetime

# ----------------------------------------------------------------
# Perp and Spot coins in Binance
perp_coins = get_binance_coins(False)
spot_coins = get_binance_coins(True)

perp_coins = [coin.upper() for coin in perp_coins]
spot_coins = [coin.upper() for coin in spot_coins]

# ----------------------------------------------------------------
max_coins = 50
results = []
files = os.listdir('../data/market_cap_snapshots')
for file in tqdm(files):
    year = int(file[:4])
    month = int(file[4:6])
    day = int(file[6:8])
    date = datetime(year, month, day)

    df = pd.read_csv(f'../data/market_cap_snapshots/{file}')

    top = ['']*max_coins
    cont = 0
    for i, row in df.iterrows():
        mask_1 = (not 'dai' in row['coin'].lower())
        mask_2 = (not 'usd' in row['coin'].lower())
        mask_3 = (not 'wbtc' in row['coin'].lower())
        mask_4 = (not 'weth' in row['coin'].lower())
        mask_5 = ((row['coin'] in spot_coins) or (row['coin'] in perp_coins))
        if mask_1 and mask_2 and mask_3 and mask_4 and mask_5:
            top[cont] = row['coin']
            cont += 1
            if cont >= max_coins:
                break

    dicc = {
        'date': date
    }
    for i, coin in enumerate(top):
        dicc[f'a{i}'] = coin
    results.append(dicc)

top_market_cap = pd.DataFrame(results)
top_market_cap['date'] = top_market_cap['date'].dt.date
top_market_cap

100%|███████████████████████████████████████████████████████████████████████████████| 145/145 [00:00<00:00, 216.45it/s]


,date,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a40,a41,a42,a43,a44,a45,a46,a47,a48,a49
0,2013-04-28,BTC,,,,,,,,,...,,,,,,,,,,
1,2013-05-05,BTC,,,,,,,,,...,,,,,,,,,,
2,2013-06-02,BTC,,,,,,,,,...,,,,,,,,,,
3,2013-07-07,BTC,,,,,,,,,...,,,,,,,,,,
4,2013-08-04,BTC,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2024-12-01,BTC,ETH,XRP,SOL,BNB,DOGE,ADA,AVAX,SHIB,...,AAVE,OP,FTM,XMR,INJ,THETA,GRT,SEI,WLD,ENA
141,2025-01-05,BTC,ETH,XRP,SOL,BNB,DOGE,ADA,TRX,AVAX,...,TIA,STX,BONK,INJ,PENGU,MOVE,IMX,THETA,FTM,GRT
142,2025-02-02,BTC,ETH,XRP,SOL,BNB,DOGE,ADA,SYS,TRX,...,ATOM,ENA,LDO,DEXE,TIA,RAY,STX,THETA,IMX,INJ
143,2025-03-02,BTC,ETH,XRP,SOL,BNB,ADA,DOGE,TRX,LINK,...,ATOM,OP,FET,DEXE,IP,ENA,INJ,IMX,STX,MKR


In [4]:
coins = []
for i, row in top_market_cap.iterrows():
    for i in range(max_coins):
        coins.append(row[f'a{i}'])

coins = list(set([coin for coin in coins if not pd.isna(coin) and coin != '']))

# ----------------------------------------------------------------
# Transform dataframe
df = {
    'date': []
}
for coin in coins:
    df[coin] = []

for i, row in top_market_cap.iterrows():
    for coin in coins:
        if coin in row.values:
            df[coin].append(1)
        else:
            df[coin].append(0)
    df['date'].append(row['date'])

top_market_cap = pd.DataFrame(df)
top_market_cap.index = range(len(top_market_cap))
top_market_cap

,date,PENGU,STRAT,HNT,POL,POWR,CRV,ENA,ADA,IOTA,...,1000SATS,OM,ENJ,ARDR,PEPE,MINA,FTM,JUP,DENT,SHIB
0,2013-04-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2013-05-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2013-06-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2013-07-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2013-08-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2024-12-01,0,0,0,1,0,0,1,1,0,...,0,1,0,0,1,0,1,0,0,1
141,2025-01-05,1,0,0,1,0,0,1,1,0,...,0,1,0,0,1,0,1,0,0,1
142,2025-02-02,0,0,0,1,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1
143,2025-03-02,0,0,0,1,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1


In [5]:
# --------------------------------------------------------
# Download historical data for all coins (only those that have been 50 and >100M$)
start_date = time.time() - 20*365*24*3600

data = get_historical_klines('BTCUSDT', start_date, '1d', True)
data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data['date'] = data['date'].dt.date
data['BTC_open'] = data['open']
data['BTC_high'] = data['high']
data['BTC_low'] = data['low']
data['BTC_close'] = data['close']
data = data[['date', 'BTC_open', 'BTC_high', 'BTC_low', 'BTC_close']]

for coin in tqdm(coins):
    if coin == 'BTC':
        continue

    try:
        if coin in spot_coins and coin in perp_coins:
            # If there is both Perp and Spot data, download both (for more completeness)
            df_spot = get_historical_klines(f'{coin}USDT', start_date, '1d', True)
            df_perp = get_historical_klines(f'{coin}USDT', start_date, '1d', False)
            
            df_spot['date'] = pd.to_datetime(df_spot['timestamp'], unit='s')
            df_spot['date'] = df_spot['date'].dt.date
            
            df_perp['date'] = pd.to_datetime(df_perp['timestamp'], unit='s')
            df_perp['date'] = df_perp['date'].dt.date
            
            df_spot = df_spot.set_index('date').sort_index()
            df_perp = df_perp.set_index('date').sort_index()
            
            df = df_perp.combine_first(df_spot).reset_index()
        elif coin in spot_coins:
            df = get_historical_klines(f'{coin}USDT', start_date, '1d', True)
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df['date'] = df['date'].dt.date
        elif coin in perp_coins:
            df = get_historical_klines(f'{coin}USDT', start_date, '1d', False)
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df['date'] = df['date'].dt.date
        else:
            print(f'{coin} not in Binance')
            continue
    except:
        print(f'ERROR: {coin}')
        continue
        
    df[f'{coin}_open'] = df['open']
    df[f'{coin}_high'] = df['high']
    df[f'{coin}_low'] = df['low']
    df[f'{coin}_close'] = df['close']
    df = df[['date', f'{coin}_open', f'{coin}_high', 
             f'{coin}_low', f'{coin}_close']]

    data = pd.merge(data, df, on='date', how='outer')
    time.sleep(3)

100%|████████████████████████████████████████████████████████████████████████████████| 185/185 [19:17<00:00,  6.26s/it]


In [6]:
data.to_csv('../data/data_50.csv')